In [1]:
import can
import matplotlib as plt
import numpy as np
import os
import time
import math
from IPython.display import display, clear_output
from joint_can import *
%matplotlib inline


In [2]:
# class JointData():
#     position=0
#     speed=0
#     torque=0
#     bearing_temp=0
#     FSM=0
#     MC_error=0
#     MC_error_occured=0
#     joint_error=0
#     joint_warning=0
#     MA_val=0
#     motor_temp=0

In [3]:
# def send_msg(adr, data, bus):
#     msg = can.Message(arbitration_id=adr, data=data, is_fd=True, is_extended_id=False)
#     try:
#         bus.send(msg,1)
#         # print(f"Message sent on {bus.channel_info}")
#     except can.CanError:
#         print("Message NOT sent")
#     return bus.recv(1)

In [4]:
# def set_speed(speed,bus):
#     speed_data=(int)(speed*32767/2)
#     speed_data=speed_data.to_bytes(2,'big',signed=True)
#     return get_data(send_msg(0x000,speed_data,bus),bus)

In [5]:
# def get_data(rcv_msg,bus):
#     data=rcv_msg.data
#     joint_data=JointData()
#     joint_data.position=float(int.from_bytes(data[0:4],'big',signed=True))*math.pi/2147483647
#     joint_data.speed=float(int.from_bytes(data[4:6],'big',signed=True))*2/32767
#     joint_data.torque=float(int.from_bytes(data[6:8],'big',signed=True))*360/32767
#     joint_data.bearing_temp=data[8]
#     joint_data.FSM=data[9]
#     joint_data.MC_error=data[10]
#     joint_data.MC_error_occured=data[11]
#     joint_data.joint_error=data[12]
#     joint_data.joint_warning=data[13]
#     joint_data.MA_val=int.from_bytes(data[14:16],'big',signed=True)
#     joint_data.motor_temp=data[16]

#     return joint_data

In [6]:
# ustawienie adresów CAN na "0"

def initialize_joint(bus):
    send_msg(0x0A0, None, bus)
    time.sleep(1)
    
    send_msg(0x0b0, [0,0,0,0,0,0], bus)
    time.sleep(0.1)
    send_msg(0x040, None, bus)
    time.sleep(0.1)
    send_msg(0x0a0, None, bus)
    time.sleep(1)
    
    if (send_msg(0x0F0, [2, 2], bus).data[0] == 1):
        send_msg(0x010, [2], bus)
        time.sleep(1)
        print(send_msg(0x010, [3], bus).data[0])
        time.sleep(5)
        return True
    else:
        return False
     

In [7]:
if __name__ == "__main__":

    os.popen('/home/ubuntu/can_down.sh')
    time.sleep(1)
    os.popen('/home/ubuntu/can_up.sh')
    time.sleep(1)

    bus = can.interface.Bus(bustype='socketcan', channel='can0', bitrate=1000000, data_bitrate=5000000, fd=True)
    initialize_joint(bus)

2


In [8]:
spin=1
it=0

In [19]:
speed=0.6 #tutaj ustawiasz predkosc w rad/s

In [20]:
# fig = plt.pyplot.figure()
# ax = fig.add_subplot(1, 1, 1) 

#torque zwracany co 1s w Nm
while(spin):
    data=set_speed(speed,bus)

    print(f'\r tq: {data.torque:.3f}\tm_temp: {data.motor_temp:.3f}\tb_temp: {data.bearing_temp:.3f}', end='')
    time.sleep(1)
    # ax.cla()
    # ax.plot(it, data.torque)
    # display(fig)
    # clear_output(wait = True)
    # it+=1
    # plt.pyplot.pause(5)
set_speed(0.,bus)

 tq: 21.050	m_temp: 33.000	b_temp: 30.000

KeyboardInterrupt: 

In [ ]:
set_speed(0.05,bus)    #zatrzymanie ctrl-enter